In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

24/11/12 17:47:00 WARN Utils: Your hostname, hcode-ThinkPad-E14-Gen-3 resolves to a loopback address: 127.0.1.1; using 192.168.1.41 instead (on interface wlp3s0)
24/11/12 17:47:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/12 17:47:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("hw_200.csv", inferSchema = True, header = True)

In [3]:
# Read the CSV file into a PySpark DataFrame and display its schema.
df.printSchema()

root
 |-- Index: integer (nullable = true)
 |--  Height(Inches)": double (nullable = true)
 |--  "Weight(Pounds)": double (nullable = true)



In [4]:
df.groupBy().avg().show()

+----------+---------------------+----------------------+
|avg(Index)|avg( Height(Inches)")|avg( "Weight(Pounds)")|
+----------+---------------------+----------------------+
|     100.5|    67.94979999999998|     127.2219500000001|
+----------+---------------------+----------------------+



In [5]:
# Explore the data: Calculate basic statistics like average height and weight.
avg_height, avg_weight = df.select(df.columns[1:]).groupBy().avg().collect()[0]
print(f'''avg height : {avg_height}
avg_weight : {avg_weight}''')

avg height : 67.94979999999998
avg_weight : 127.2219500000001


In [6]:
# Filter the DataFrame to include only rows where the weight is above the average weight.
df[df[" \"Weight(Pounds)\""] > avg_weight].show()

+-----+----------------+-----------------+
|Index| Height(Inches)"| "Weight(Pounds)"|
+-----+----------------+-----------------+
|    2|           71.52|           136.49|
|    3|            69.4|           153.03|
|    4|           68.22|           142.34|
|    5|           67.79|            144.3|
|    7|            69.8|           141.49|
|    8|           70.01|           136.46|
|   11|           66.49|           127.45|
|   16|           71.09|            140.0|
|   17|           66.46|            129.5|
|   18|           68.65|           142.97|
|   19|           71.23|            137.9|
|   21|           67.83|           141.28|
|   22|           68.88|           143.54|
|   24|           68.42|            129.5|
|   25|           67.63|           141.85|
|   26|           67.21|           129.72|
|   27|           70.84|           142.42|
|   28|           67.49|           131.55|
|   34|            70.6|           136.22|
|   35|            71.8|            140.1|
+-----+----

In [7]:
# Add a new column 'BMI' (Body Mass Index) using the formula: BMI = weight (kg) / (height (m))^2.
df_with_bmi = df.withColumn("BMI", (df[" \"Weight(Pounds)\""] / df[" Height(Inches)\""]) ** 2)
df_with_bmi.show()

+-----+----------------+-----------------+------------------+
|Index| Height(Inches)"| "Weight(Pounds)"|               BMI|
+-----+----------------+-----------------+------------------+
|    1|           65.78|           112.99| 2.950477308987891|
|    2|           71.52|           136.49| 3.642056308305683|
|    3|            69.4|           153.03| 4.862215635874393|
|    4|           68.22|           142.34| 4.353419245390666|
|    5|           67.79|            144.3|  4.53107649746846|
|    6|            68.7|            123.3| 3.221162830609637|
|    7|            69.8|           141.49| 4.109042639222996|
|    8|           70.01|           136.46| 3.799186195797401|
|    9|            67.9|           112.37|2.7388056376764753|
|   10|           66.78|           120.67|3.2651699881988576|
|   11|           66.49|           127.45| 3.674235709439762|
|   12|           67.62|           114.14|2.8492161907263833|
|   13|            68.3|           125.61|3.3822602676590448|
|   14| 

In [9]:
# Save the transformed DataFrame as a Parquet file for efficient storage.
df_with_bmi.write.parquet("hw_200.parquet")

In [10]:
# Testing whether file saved correctly 
test_parquet = spark.read.parquet("hw_200.parquet")
test_parquet.show()

+-----+----------------+-----------------+------------------+
|Index| Height(Inches)"| "Weight(Pounds)"|               BMI|
+-----+----------------+-----------------+------------------+
|    1|           65.78|           112.99| 2.950477308987891|
|    2|           71.52|           136.49| 3.642056308305683|
|    3|            69.4|           153.03| 4.862215635874393|
|    4|           68.22|           142.34| 4.353419245390666|
|    5|           67.79|            144.3|  4.53107649746846|
|    6|            68.7|            123.3| 3.221162830609637|
|    7|            69.8|           141.49| 4.109042639222996|
|    8|           70.01|           136.46| 3.799186195797401|
|    9|            67.9|           112.37|2.7388056376764753|
|   10|           66.78|           120.67|3.2651699881988576|
|   11|           66.49|           127.45| 3.674235709439762|
|   12|           67.62|           114.14|2.8492161907263833|
|   13|            68.3|           125.61|3.3822602676590448|
|   14| 